<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_ResNet/test_sample_ResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 1137 (delta 93), reused 133 (delta 48), pack-reused 938
Receiving objects: 100% (1137/1137), 8.50 MiB | 21.02 MiB/s, done.
Resolving deltas: 100% (664/664), done.


In [2]:
%cd PyTorch-Architectures/modeling_ResNet/

/content/PyTorch-Architectures/modeling_ResNet


In [7]:
# TODO --> paper needs to be read carefully
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets

from model_ResNet18 import ResNet, BasicBlock

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [4]:
# Hyper parameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001 
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 28 * 28
NUM_CLASSES = 10

# Other
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = True

In [ ]:
##################
# MNIST Dataset
##################

train_dataset = datasets.MNIST(
    root='data',
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
test_dataset = datasets.MNIST(
    root='data',
    train=False,
    transform=transforms.ToTensor(),
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    )
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

In [6]:
# Checking the dataset
for images, labels in train_loader:
  print('Image Batch Dimensions: ', images.shape)
  print('Label Batch Dimensions: ', labels.shape)
  break


Image Batch Dimensions:  torch.Size([128, 1, 28, 28])
Label Batch Dimensions:  torch.Size([128])


In [8]:
def resnet18(num_classes):
  model = ResNet(block=BasicBlock,
                 layers=[2, 2, 2, 2],
                 num_classes=NUM_CLASSES,
                 grayscale=grayscale)
  return model

In [13]:
torch.manual_seed(RANDOM_SEED)

model = resnet18(NUM_CLASSES)
model.to(device)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters: ", params)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Total Trainable Parameters:  11175370


In [ ]:
def compute_accuracy(model, data_loader, device):
  pass

start_time = time.time()
for epoch in range(NUM_EPOCHS):
  model.train()
  for batch_idx, (features, targets) in enumerate(train_loader):
    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    cost = F.cross_entropy(logits, targets)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    #LOGGING
    if batch_idx % 50 == 0:
      print('Batch: %04d/%04d | Epoch: %04d/%04d | Cost: %.4f' % (batch_idx, len(train_loader), epoch+1, NUM_EPOCHS, cost.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    print('Epoch: %04d/%04d | Train %.3f%%' % (epoch+1, NUM_EPOCHS, compute_accuracy(model, test_loader, device=device)))
  
  print("Epoch elapsed time %.2f min" % ((time.time() - start_time) / 60))
print('Total Training Time: %.2f min' % ((time.time() - start_time) / 60))